# **SchNetPack Ensemble Calculator for Atomistic Simulations**

This tutorial demonstrates how to use SchNetPack’s ensemble calculator to predict atomic energies and forces with uncertainty estimation.

We’ll walk through the following examples:
- How to calculate ensemble-based uncertainty
- Structure relaxation using ensemble predictions
- Running molecular dynamics (MD) simulations with uncertainty

These tools are useful for identifying uncertain regions in simulation trajectories and making more informed decisions in atomistic modeling.


In [ ]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from ase import units
from ase.io import read
from ase.optimize.lbfgs import LBFGS
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin

from schnetpack.interfaces.ase_interface import SpkEnsembleCalculator, AbsoluteUncertainty, RelativeUncertainty
import schnetpack.transform as trn
from schnetpack.datasets import MD17

In [ ]:
#load data into atoms object
atoms = read('../../tests/testdata/md_ethanol.xyz', index=0)

## Ensemble Interface to ASE

We specify a list of pre-trained SchNet models trained on ethanol data. These models constitute the **ensemble**, which will be used to serve as a testbed for SchNetPack’s ensemble-enabled `SpkEnsembleCalculator`.

In [ ]:
model_path_list = ['../../tests/testdata/md_ethanol.model',
                   '../../tests/testdata/md_ethanol_2.model']

### ⚖️ Creating an Ensemble Calculator with Uncertainty Quantification

In this section, we instantiate two different uncertainty estimators:
- `AbsoluteUncertainty` calculates raw standard deviation values.
- `RelativeUncertainty` gives uncertainty as a fraction of the mean, which helps when comparing predictions with different scales.

Both uncertainty methods are bundled together in `SpkEnsembleCalculator`. This lets us evaluate uncertainty in multiple ways for the same prediction run, giving a more complete picture of model confidence.

Finally, we create the `SpkEnsembleCalculator`, which uses multiple trained models to make predictions. It also estimates uncertainty using the methods we provided. This calculator will act just like a regular ASE calculator but with built-in support for ensemble averaging and uncertainty tracking.

In [ ]:
uncertainty_abs = AbsoluteUncertainty(energy_weight=0.5,force_weight=1.0)
uncertainty_rel = RelativeUncertainty(energy_weight=1.0, force_weight=2.0)

uncertainty = [uncertainty_abs, uncertainty_rel]

ensemble_calculator = SpkEnsembleCalculator(
    models=model_path_list,
    neighbor_list=trn.ASENeighborList(cutoff=5.0),
    energy_key=MD17.energy,
    force_key=MD17.forces,
    energy_unit="kcal/mol",
    position_unit="Ang",
    uncertainty_fn=uncertainty)

Assign the ensemble calculator `ensemble_calculator` to the atoms object

In [ ]:
atoms.calc = ensemble_calculator

 🔮 Prediction Output:
- ⚡ Energy: Total potential energy of the atomic system.
- 🔧 Forces: Atomic forces (gradients of energy) for optimization or dynamics.
- 📊 Uncertainty: Estimation of model prediction uncertainty from the ensemble.

In [ ]:
print("Prediction:")
print("energy:", atoms.get_total_energy())
print("forces:", atoms.get_forces())
print("uncertainty:", ensemble_calculator.get_uncertainty(atoms))

# Structure Optimization

🏗️ **Distort molecular structure:**
- A small random disturbance is added to the atomic positions.
- This simulates a noisy or slightly perturbed structure, which helps us test how sensitive the ensemble model is to input changes.
- It also makes the uncertainty values more meaningful by introducing some variability.

In [ ]:
# distort the structure
atoms.positions += np.random.normal(0, 0.3, atoms.positions.shape)

⚙️ **Set Up Optimization:**
- 🧑‍🔬 **Optimizer**: We set up the LBFGS optimizer, which is a gradient-based method used to minimize the energy of the atomic system.
- 🔬 **Calculator**: Assign the ensemble calculator to the atoms object, this connects the prediction engine (our ensemble of models) to the atomic system.

In [ ]:
optimizer = LBFGS(atoms)
atoms.calc = ensemble_calculator

🔄 **Optimization Loop with Uncertainty Tracking:**
- 🧑‍🔬 **Optimizer**: Run the optimization using the LBFGS algorithm with a force tolerance of 0.01 and a maximum of 100 steps.
- 📊 **Uncertainty Tracking**: After each optimization step, the uncertainty of the energy prediction is appended to the `uncertainties` list, providing insight into model confidence during the process.

In [ ]:
uncertainties = []

for _ in optimizer.irun(fmax=0.05, steps=300):
    uncertainties.append(ensemble_calculator.get_uncertainty(atoms))

- **Absolute** and **Relative Uncertainty** values are extracted from the optimization steps.
- Plot both **absolute** and **relative** uncertainties against the optimization steps to visualize how uncertainty changes during the process.

In [ ]:
# Extract individual uncertainty types
abs_vals = [d["AbsoluteUncertainty"] for d in uncertainties]
rel_vals = [d["RelativeUncertainty"] for d in uncertainties]
steps = list(range(len(uncertainties)))

# Create figure and first axis
fig, ax1 = plt.subplots(figsize=(8, 6))

# Plot absolute uncertainty on left y-axis
ax1.plot(steps, abs_vals, label="Absolute Uncertainty", marker='o', color='tab:blue')
ax1.set_xlabel("Optimization Step")
ax1.set_ylabel("Absolute Uncertainty", color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')
ax1.grid(True)

# Create second y-axis for relative uncertainty
ax2 = ax1.twinx()
ax2.plot(steps, rel_vals, label="Relative Uncertainty", marker='x', color='tab:red')
ax2.set_ylabel("Relative Uncertainty", color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

# Title and layout
plt.title("Uncertainty during Optimization")
fig.tight_layout()
plt.show()


Since we're using an ensemble of models, we can now estimate the uncertainty in our predictions during the optimization process.
While the absolute uncertainty remains low and stable throughout, the relative uncertainty shows sharp fluctuations at specific steps.
This indicates that although the model is generally confident in its predictions, there are certain configurations where the predictions become less reliable relative to their magnitude.
These fluctuations can serve as useful indicators of difficult or out-of-distribution structures during optimization.

# Molecular Dynamics With Annealing Temperature

In this setup, we use only absolute uncertainty to measure how much the model predictions vary across the ensemble:

🔎 **Note**: The `uncertainty_fn` can be passed as either a **single** uncertainty function or as a **list** of uncertainty functions, depending on the desired level of complexity.


In [ ]:
uncertainty_abs = AbsoluteUncertainty(energy_weight=0.5,force_weight=1.0)

abs_ensemble_calculator = SpkEnsembleCalculator(
    models=model_path_list,
    neighbor_list=trn.ASENeighborList(cutoff=5.0),
    energy_key=MD17.energy,
    force_key=MD17.forces,
    energy_unit="kcal/mol",
    position_unit="Ang",
    uncertainty_fn=uncertainty_abs)

In [ ]:
atoms.calc = ensemble_calculator

## Temperature Annealing and Langevin Dynamics with SchNetPack

In this simulation, we apply **temperature annealing** and **Langevin dynamics** to model the behavior of ethanol under varying temperatures, optimizing molecular configurations and simulating thermal effects.

### Annealing

Annealing gradually cools the system from a high initial temperature ($T_{\text{start}}$) to a lower final temperature ($T_{\text{end}}$) over time. The temperature is decayed exponentially:

$$
T_{\text{annealed}}(t) = T_{\text{start}} \times \exp\left(-\frac{t}{n_{\text{steps}}} \times \ln\left(\frac{T_{\text{start}}}{T_{\text{end}}}\right)\right)
$$

This cooling schedule helps the system avoid local minima and settle into a more stable configuration.

### Langevin Dynamics

Langevin dynamics includes both deterministic forces and stochastic noise to simulate the interaction with a thermal bath. The equation of motion for atom $i$ is:

$$
m_i \frac{d^2 \mathbf{r}_i}{dt^2} = \mathbf{F}_i - \gamma m_i \frac{d \mathbf{r}_i}{dt} + \sqrt{2 \gamma k_B T} \xi_i(t)
$$

Where:
- $\mathbf{F}_i$ is the force acting on atom $i$,
- $\gamma$ is the friction coefficient,
- $m_i$ is the mass of the atom,
- $T$ is the temperature, and
- $\xi_i(t)$ is a random Gaussian noise term.

This term controls the system's energy dissipation and allows thermal fluctuations, mimicking a heat bath.

### Uncertainty Estimation with Machine Learning Potentials

SchNetPack uses machine learning models to predict energy, forces, and uncertainties. The uncertainty is calculated from the variation between different models in the ensemble, giving a measure of confidence in the predictions.

By combining **annealing**, **Langevin dynamics**, and **uncertainty estimation**, we can simulate molecular systems with thermal fluctuations and optimize configurations. The use of machine learning potentials enables more accurate and efficient simulations, particularly for large systems where classical methods may be less reliable.


In [ ]:
# Annealing parameters
temperature_start = 500  
temperature_end = 50    
n_steps = 1000          
sampling_interval = 10  
step_size = 0.5         

# Cooling schedule function: Exponential decay of temperature
def get_annealed_temperature(step, n_steps, start_temp, end_temp):
    return start_temp * np.exp(-step / n_steps * np.log(start_temp / end_temp))

MaxwellBoltzmannDistribution(atoms, temperature_K=temperature_start)

dyn = Langevin(
    atoms, 
    timestep=step_size * units.fs, 
    temperature_K=temperature_start,
    friction=0.001 / units.fs
)

ats_traj = []
uncertainties = []
e_tmp = []

for step in tqdm(range(n_steps // sampling_interval)):
    annealed_temperature = get_annealed_temperature(step, n_steps, temperature_start, temperature_end)

    dyn.temperature_K = annealed_temperature

    dyn.run(sampling_interval)

    e_tmp.append(atoms.get_temperature())
    uncertainties.append(abs_ensemble_calculator.get_uncertainty(atoms))
    
    ats_traj.append(atoms.copy())

    print(f"Step {step * sampling_interval}, Annealed Temp: {annealed_temperature:.2f} K")

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 6))

ax1.plot(uncertainties, marker='o', color='blue', label='Uncertainty')
ax1.set_xlabel("MD Step")
ax1.set_ylabel("Uncertainty", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(e_tmp, marker='x', color='red', label='Temperature')
ax2.set_ylabel("Temperature (K)", color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title("Molecular Dynamics: Uncertainty and Temperature Profile")
ax1.grid(True)

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right')

plt.tight_layout()
plt.show()

This annealing MD simulation cools the system from 500 K to 50 K via an exponential decay schedule. The temperature profile (red) reflects this trend with expected thermal fluctuations.

Throughout the trajectory, the ensemble model's uncertainty (blue) remains stable despite the changing thermodynamic conditions. This indicates robust generalization of the model across varying configurations, ensuring reliable predictions even as the system explores lower-energy states.